# Analyze Tournament Site Location Data

## Import data and packages

In [1]:
# import python packages
import pandas as pd

# import school distances dataset
data = pd.read_csv('../data/cleaned/distances-schools.csv')
data.head()

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance
0,1.0,Duke,"Columbia, SC",2019.0,20190,Duke University,Blue Devils,Durham,North Carolina,Private/Non-sectarian,Atlantic Coast Conference,Duke University Durham North Carolina,-78.944230,36.000156,POINT (-78.94422972195878 36.00015569999999),179.765685
1,1.0,Gonzaga,"Salt Lake City, UT",2019.0,20191,Gonzaga University,Bulldogs,Spokane,Washington,Private/Catholic,West Coast Conference,Gonzaga University Spokane Washington,-117.403044,47.666739,POINT (-117.4030438539681 47.66673855000001),549.380264
2,1.0,North Carolina,"Columbus, OH",2019.0,20192,University of North Carolina at Chapel Hill,Tar Heels,Chapel Hill,North Carolina,State,Atlantic Coast Conference,University of North Carolina at Chapel Hill Ch...,-79.047753,35.905035,POINT (-79.04775326525106 35.90503535),352.052893
3,1.0,Virginia,"Columbia, SC",2019.0,20193,University of Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference,University of Virginia Charlottesville Virginia,-78.505500,38.041058,POINT (-78.50549960183569 38.0410576),296.237023
4,2.0,Michigan State,"Des Moines, IA",2019.0,20194,Michigan State University,Spartans,East Lansing,Michigan,State,Big Ten Conference,Michigan State University East Lansing Michigan,-84.477916,42.718568,POINT (-84.47791570930522 42.71856800000001),571.484627


## Find weighted distance

In theory, higher seeded teams should play at closer sites, and the further down the list of the top 16 teams, the lesser geographic preference. To compare higher and lower seeds on the same level, a simple weighted average is calculated where 1 seeds are weighted 1, 2 seeds are weighted 0.75, 3 seeds are weighted 0.5, and 4 seeds are weighted 0.25.

In [2]:
# dictionary of weights - seeds are keys, weights are values
weights = {1: 1, 2: 0.75, 3: 0.5, 4: 0.25}

# loop through distances and apply weights based on the associated seed value 
weightedDistance = [dist * weights[data.seed[i]] for i, dist in enumerate(data.distance)]

# add weighted distance column to dataframe
data['weightedDist'] = weightedDistance
data.tail()

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance,weightedDist
555,3.0,NC State,"Albuquerque, NM",1985.0,1985555,North Carolina State University,Wolfpack,Raleigh,North Carolina,State,Atlantic Coast Conference,North Carolina State University Raleigh North ...,-78.674087,35.771850,POINT (-78.67408695452633 35.77184965),1739.300819,869.650410
556,4.0,Loyola–Chicago,"Hartford, CT",1985.0,1985556,Loyola University Chicago,Ramblers,Chicago,Illinois,Private/Catholic,Missouri Valley Conference,Loyola University Chicago Chicago Illinois,-87.668422,41.944842,POINT (-87.66842176669064 41.94484179999999),930.929259,232.732315
557,4.0,Ohio State,"Tulsa, OK",1985.0,1985557,The Ohio State University,Buckeyes,Columbus,Ohio,State,Big Ten Conference,The Ohio State University Columbus Ohio,-83.028663,40.005709,POINT (-83.02866259769122 40.00570905),840.512615,210.128154
558,4.0,LSU,"Dayton, OH",1985.0,1985558,Louisiana State University,Tigers,Baton Rouge,Louisiana,State,Southeastern Conference,Louisiana State University Baton Rouge Louisiana,-91.185968,30.405709,POINT (-91.18596767189877 30.40570885),725.868192,181.467048
559,4.0,UNLV,"Salt Lake City, UT",1985.0,1985559,"University of Nevada, Las Vegas",Rebels,Paradise,Nevada,State,Mountain West Conference,UNLV Paradise Nevada,-115.141832,36.107155,POINT (-115.1418318610852 36.1071554),352.895410,88.223853


In [3]:
# data.loc[data.school_common_name == 'Kentucky']

## Aggregate distances by school

While the distance dataframe is useful as is, it is most interesting to aggregate the data at different levels. Of most interest is aggregating at the school level. By using the Pandas `describe` method, the mean, standard deviation, minimum, maximum, and quantiles are quickly calculated for each school.

In [4]:
# group dataframe by schools and apply describe method
schoolsWeighted = data.groupby('school_common_name').describe()

# preserve the weighted distance aggregation only
schoolsWtDistAgg = schoolsWeighted.weightedDist
schoolsWtDistAgg.head()

,count,mean,std,min,25%,50%,75%,max
school_common_name,,,,,,,,
Alabama,3.0,112.809325,120.297037,37.254071,43.447920,49.641768,150.586951,251.532134
Arizona,20.0,452.091798,252.612485,0.387824,339.977205,399.902145,620.471038,828.782889
Arkansas,7.0,277.658557,168.427111,105.722508,194.128868,216.471203,304.079252,624.999944
Auburn,2.0,469.969893,31.197330,447.910049,458.939971,469.969893,480.999815,492.029737
BYU,2.0,321.585878,161.017824,207.729083,264.657480,321.585878,378.514275,435.442673


In [5]:
# preserve the distance aggregation only
schoolsDistAgg = schoolsWeighted.distance
schoolsDistAgg.head()

,count,mean,std,min,25%,50%,75%,max
school_common_name,,,,,,,,
Alabama,3.0,194.538449,142.894640,49.672095,124.119584,198.567073,266.971626,335.376179
Arizona,20.0,723.999610,442.918415,0.387824,463.827810,648.429761,859.110202,1655.073496
Arkansas,7.0,548.434711,215.707104,210.878249,422.890031,581.981962,667.258947,865.884814
Auburn,2.0,1208.014498,1074.950020,447.910049,827.962274,1208.014498,1588.066722,1968.118947
BYU,2.0,1078.614428,937.844581,415.458165,747.036297,1078.614428,1410.192560,1741.770691


## Add tags to column names to differentiate between weighted and unweighted statistics

Since both the weighted and unweighted distances were aggregated and have the same `description()` column names, tags can be concatenated to the columns to differentiate the two.

In [6]:
# concatenate `_wtDist` to each column and apply to weighted distance dataframe
schoolsWtDistAgg.columns = [col + '_wtDist' for col in schoolsWtDistAgg.columns]
display(schoolsWtDistAgg.head())

# concatenate `_dist` to each column and apply to UNweighted distance dataframe
schoolsDistAgg.columns = [col + '_dist' for col in schoolsDistAgg.columns]
schoolsDistAgg.head()

,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
school_common_name,,,,,,,,
Alabama,3.0,112.809325,120.297037,37.254071,43.447920,49.641768,150.586951,251.532134
Arizona,20.0,452.091798,252.612485,0.387824,339.977205,399.902145,620.471038,828.782889
Arkansas,7.0,277.658557,168.427111,105.722508,194.128868,216.471203,304.079252,624.999944
Auburn,2.0,469.969893,31.197330,447.910049,458.939971,469.969893,480.999815,492.029737
BYU,2.0,321.585878,161.017824,207.729083,264.657480,321.585878,378.514275,435.442673


,count_dist,mean_dist,std_dist,min_dist,25%_dist,50%_dist,75%_dist,max_dist
school_common_name,,,,,,,,
Alabama,3.0,194.538449,142.894640,49.672095,124.119584,198.567073,266.971626,335.376179
Arizona,20.0,723.999610,442.918415,0.387824,463.827810,648.429761,859.110202,1655.073496
Arkansas,7.0,548.434711,215.707104,210.878249,422.890031,581.981962,667.258947,865.884814
Auburn,2.0,1208.014498,1074.950020,447.910049,827.962274,1208.014498,1588.066722,1968.118947
BYU,2.0,1078.614428,937.844581,415.458165,747.036297,1078.614428,1410.192560,1741.770691


## Append distance and weighted distance dataframes

Because both of the aggregations will be merged into the `data` dataframe, they should be appended together in one dataframe. They can be merged by their matching index keys (school names).

In [7]:
# append two dataframes into one
aggs = pd.merge(schoolsDistAgg, schoolsWtDistAgg, left_on=schoolsDistAgg.index.get_level_values('school_common_name'), right_on=schoolsWtDistAgg.index.get_level_values('school_common_name'))
aggs.head()

,key_0,count_dist,mean_dist,std_dist,min_dist,25%_dist,50%_dist,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
0,Alabama,3.0,194.538449,142.894640,49.672095,124.119584,198.567073,266.971626,335.376179,3.0,112.809325,120.297037,37.254071,43.447920,49.641768,150.586951,251.532134
1,Arizona,20.0,723.999610,442.918415,0.387824,463.827810,648.429761,859.110202,1655.073496,20.0,452.091798,252.612485,0.387824,339.977205,399.902145,620.471038,828.782889
2,Arkansas,7.0,548.434711,215.707104,210.878249,422.890031,581.981962,667.258947,865.884814,7.0,277.658557,168.427111,105.722508,194.128868,216.471203,304.079252,624.999944
3,Auburn,2.0,1208.014498,1074.950020,447.910049,827.962274,1208.014498,1588.066722,1968.118947,2.0,469.969893,31.197330,447.910049,458.939971,469.969893,480.999815,492.029737
4,BYU,2.0,1078.614428,937.844581,415.458165,747.036297,1078.614428,1410.192560,1741.770691,2.0,321.585878,161.017824,207.729083,264.657480,321.585878,378.514275,435.442673


## Prep original `data` dataframe for merging

The weighted and unweighted distance aggregations need to be merged with the main school dataframe. Because the aggregations are at the school level, we only need the generic school information columns - the `year`, `seed`, `site`, `id`, and `geometry` columns can be dropped from `data`. Furthermore, all duplicate entries can be dropped.



In [8]:
# drop all duplicate school names, drop columns that change depending on the year
schoolsTrimmed = data.drop_duplicates(['school_common_name']).drop(['year', 'seed', 'site', 'id', 'geometry', 'distance', 'weightedDist'], axis=1)
schoolsTrimmed.head()

,school_common_name,school_full_name,team,city,state,type,conference,address,lng,lat
0,Duke,Duke University,Blue Devils,Durham,North Carolina,Private/Non-sectarian,Atlantic Coast Conference,Duke University Durham North Carolina,-78.944230,36.000156
1,Gonzaga,Gonzaga University,Bulldogs,Spokane,Washington,Private/Catholic,West Coast Conference,Gonzaga University Spokane Washington,-117.403044,47.666739
2,North Carolina,University of North Carolina at Chapel Hill,Tar Heels,Chapel Hill,North Carolina,State,Atlantic Coast Conference,University of North Carolina at Chapel Hill Ch...,-79.047753,35.905035
3,Virginia,University of Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference,University of Virginia Charlottesville Virginia,-78.505500,38.041058
4,Michigan State,Michigan State University,Spartans,East Lansing,Michigan,State,Big Ten Conference,Michigan State University East Lansing Michigan,-84.477916,42.718568


In [9]:
# merge schools with weights distance aggregation on
schools = pd.merge(schoolsTrimmed, aggs, how='left', left_on='school_common_name', right_on='key_0')
display(schools.head())

# sort by mean weighted distance so that higher means (and thus bigger proportional circles) are plotted first
schoolsSorted = schools.sort_values(by=['mean_wtDist'], ascending=False)
schoolsSorted.head()

,school_common_name,school_full_name,team,city,state,type,conference,address,lng,lat,...,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
0,Duke,Duke University,Blue Devils,Durham,North Carolina,Private/Non-sectarian,Atlantic Coast Conference,Duke University Durham North Carolina,-78.944230,36.000156,...,367.019606,2068.651965,31.0,208.909988,231.936620,6.594143,52.879953,127.422660,240.599142,1034.325982
1,Gonzaga,Gonzaga University,Bulldogs,Spokane,Washington,Private/Catholic,West Coast Conference,Gonzaga University Spokane Washington,-117.403044,47.666739,...,549.380264,1039.667928,9.0,339.566741,203.415695,65.642776,229.680863,274.690132,549.380264,549.380264
2,North Carolina,University of North Carolina at Chapel Hill,Tar Heels,Chapel Hill,North Carolina,State,Atlantic Coast Conference,University of North Carolina at Chapel Hill Ch...,-79.047753,35.905035,...,476.946658,2786.543099,26.0,331.367343,407.281304,26.553060,78.860877,159.503568,410.998203,1547.355379
3,Virginia,University of Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference,University of Virginia Charlottesville Virginia,-78.505500,38.041058,...,279.018491,369.170067,6.0,177.984456,72.930227,92.292517,140.746063,155.634117,213.152715,296.237023
4,Michigan State,Michigan State University,Spartans,East Lansing,Michigan,State,Big Ten Conference,Michigan State University East Lansing Michigan,-84.477916,42.718568,...,621.912538,2070.326087,12.0,294.073988,176.723598,38.663768,187.515849,271.869960,423.238462,584.228041


,school_common_name,school_full_name,team,city,state,type,conference,address,lng,lat,...,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
73,St. John's,St. John's University,Red Storm,Jamaica,New York,Private/Catholic,Big East Conference,St. John's University Jamaica New York,-73.990073,40.729944,...,2358.641641,2780.999650,5.0,1501.481643,1157.579618,159.223749,442.906099,1768.981231,2355.297484,2780.999650
85,VCU,Virginia Commonwealth University,Rams,Richmond,Virginia,State,Atlantic 10 Conference,Virginia Commonwealth University Richmond Virg...,-77.453064,37.548215,...,1821.113346,1821.113346,1.0,1365.835010,NaN,1365.835010,1365.835010,1365.835010,1365.835010,1365.835010
55,Stanford,Stanford University,Cardinal,Palo Alto,California,Private/Non-Sectarian,Pac-12 Conference,Stanford University Palo Alto California,-122.169365,37.431314,...,2176.374000,2635.125228,8.0,811.707855,696.429024,173.194292,364.331846,553.266875,1141.144455,2211.465583
70,USC,University of Southern California,Trojans,Los Angeles,California,Private/Non-Sectarian,Pac-12 Conference,University of Southern California Los Angeles ...,-118.285867,34.021883,...,1562.989869,1968.481379,2.0,781.494935,982.689063,86.628835,434.061885,781.494935,1128.927985,1476.361035
79,Seton Hall,Seton Hall University,Pirates,South Orange,New Jersey,Private/Catholic,Big East Conference,Seton Hall University South Orange New Jersey,-74.246858,40.743372,...,2340.323155,2343.279641,4.0,778.182929,505.062064,112.649327,522.242636,914.221284,1170.161577,1171.639820


In [10]:
# Find overall mean for mapping purposes
print(schoolsSorted['mean_wtDist'].mean())
schoolsSorted['mean_dist'].mean()

345.88513858848495


724.6294584016067

## Write to CSV

In [11]:
schoolsSorted.to_csv('../data/cleaned/schools-wtAvg.csv', index=False)

## Group data by school and seed

To provide a view for each school's seeding data, the data should be grouped by both the school name and seed. The `describe` method calculates various statistics for the datasets. The weighted and unweighted distances should then be joined together in one dataset.

In [12]:
# group by school and seed, calculate averages with describe method
seeds = data.groupby(['school_common_name', 'seed']).describe()
seedsDist = seeds.distance
seedsWtDist = seeds.weightedDist

display(seedsDist.head())
display(seedsWtDist.head())

count        mean         std         min  \
school_common_name seed                                              
Alabama            2.0     2.0  192.524137  202.023295   49.672095   
                   4.0     1.0  198.567073         NaN  198.567073   
Arizona            1.0     6.0  487.863183  286.797591    0.387824   
                   2.0     7.0  718.174996  279.259933  458.502104   
                   3.0     4.0  712.841103  689.426122   95.472823   

                                25%         50%         75%          max  
school_common_name seed                                                   
Alabama            2.0   121.098116  192.524137  263.950158   335.376179  
                   4.0   198.567073  198.567073  198.567073   198.567073  
Arizona            1.0   405.640533  499.402953  706.043212   778.675678  
                   2.0   533.202860  533.202860  932.035218  1105.043853  
                   3.0   265.475016  550.409046  997.775133  1655.073496

count        mean         std         min  \
school_common_name seed                                              
Alabama            2.0     2.0  144.393103  151.517471   37.254071   
                   4.0     1.0   49.641768         NaN   49.641768   
Arizona            1.0     6.0  487.863183  286.797591    0.387824   
                   2.0     7.0  538.631247  209.444950  343.876578   
                   3.0     4.0  356.420551  344.713061   47.736411   

                                25%         50%         75%         max  
school_common_name seed                                                  
Alabama            2.0    90.823587  144.393103  197.962618  251.532134  
                   4.0    49.641768   49.641768   49.641768   49.641768  
Arizona            1.0   405.640533  499.402953  706.043212  778.675678  
                   2.0   399.902145  399.902145  699.026413  828.782889  
                   3.0   132.737508  275.204523  498.887566  827.536748

### Join weighted and unweighted dataframes together

The weighted and unweighted distance dataframes should be joined together. Because the dataframes need to be joined by two matching columns (`school_common_name` and `seed`), it's easier to simply create new columns in the `seedsDist` dataframe with the columns from the `seedsWtDist` dataframe.

In [13]:
# turn off copy warning
pd.options.mode.chained_assignment = None  # default='warn'

seedsDist['mean_wtDist'] = seedsWtDist['mean']
seedsDist['std_wtDist'] = seedsWtDist['std']
seedsDist['min_wtDist'] = seedsWtDist['min']
seedsDist['25%_wtDist'] = seedsWtDist['25%']
seedsDist['50%_wtDist'] = seedsWtDist['50%']
seedsDist['75%_wtDist'] = seedsWtDist['75%']
seedsDist['max_wtDist'] = seedsWtDist['max']
seedsDist

count         mean         std          min  \
school_common_name seed                                                
Alabama            2.0     2.0   192.524137  202.023295    49.672095   
                   4.0     1.0   198.567073         NaN   198.567073   
Arizona            1.0     6.0   487.863183  286.797591     0.387824   
                   2.0     7.0   718.174996  279.259933   458.502104   
                   3.0     4.0   712.841103  689.426122    95.472823   
...                        ...          ...         ...          ...   
Wisconsin          4.0     3.0  1202.786541  287.506888   926.712988   
Xavier             1.0     1.0   234.536763         NaN   234.536763   
                   2.0     1.0   359.960356         NaN   359.960356   
                   3.0     2.0   348.427564  161.065915   234.536763   
                   4.0     1.0  1991.467245         NaN  1991.467245   

                                 25%          50%          75%          max  \
school_common_name seed                                                       
Alabama            2.0    121.098116   192.524137   263.950158   335.376179   
                   4.0    198.567073   198.567073   198.567073   198.567073   
Arizona            1.0    405.640533   499.402953   706.043212   778.675678   
                   2.0    533.202860   533.202860   932.035218  1105.043853   
                   3.0    265.475016   550.409046   997.775133  1655.073496   
...                              ...          ...          ...          ...   
Wisconsin          4.0   1053.928069  1181.143151  1340.823317  1500.503484   
Xavier             1.0    234.536763   234.536763   234.536763   234.536763   
                   2.0    359.960356   359.960356   359.960356   359.960356   
                   3.0    291.482163   348.427564   405.372964   462.318364   
                   4.0   1991.467245  1991.467245  1991.467245  1991.467245   

                         mean_wtDist  std_wtDist  min_wtDist  25%_wtDist  \
school_common_name seed                                                    
Alabama            2.0    144.393103  151.517471   37.254071   90.823587   
                   4.0     49.641768         NaN   49.641768   49.641768   
Arizona            1.0    487.863183  286.797591    0.387824  405.640533   
                   2.0    538.631247  209.444950  343.876578  399.902145   
                   3.0    356.420551  344.713061   47.736411  132.737508   
...                              ...         ...         ...         ...   
Wisconsin          4.0    300.696635   71.876722  231.678247  263.482017   
Xavier             1.0    234.536763         NaN  234.536763  234.536763   
                   2.0    269.970267         NaN  269.970267  269.970267   
                   3.0    174.213782   80.532957  117.268382  145.741082   
                   4.0    497.866811         NaN  497.866811  497.866811   

                         50%_wtDist  75%_wtDist  max_wtDist  
school_common_name seed                                      
Alabama            2.0   144.393103  197.962618  251.532134  
                   4.0    49.641768   49.641768   49.641768  
Arizona            1.0   499.402953  706.043212  778.675678  
                   2.0   399.902145  699.026413  828.782889  
                   3.0   275.204523  498.887566  827.536748  
...                             ...         ...         ...  
Wisconsin          4.0   295.285788  335.205829  375.125871  
Xavier             1.0   234.536763  234.536763  234.536763  
                   2.0   269.970267  269.970267  269.970267  
                   3.0   174.213782  202.686482  231.159182  
                   4.0   497.866811  497.866811  497.866811  

[227 rows x 15 columns]

## Calculate overall seed averages

As a point of reference, the weighted and unweighted statistics for the overall seeds (independent of schools) should be calculated.

In [14]:
# group data by seed
seedsAll = data.groupby('seed').describe()

# merge weighted and unweighted averages on 'seed', add custom suffixes to differentiate between the two
seedsMerged = pd.merge(seedsAll.distance, seedsAll.weightedDist, 
               left_on=seedsAll.distance.index.get_level_values('seed'), 
               right_on=seedsAll.weightedDist.index.get_level_values('seed'),
               suffixes=('_dist', '_wtDist'))

# rename `key_0` column as `seed` for clarity
seedsMerged['seed'] = seedsMerged['key_0']
seedsMerged

,key_0,count_dist,mean_dist,std_dist,min_dist,25%_dist,50%_dist,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist,seed
0,1.0,140.0,376.468191,450.363472,0.387824,127.422660,243.041596,451.633110,2780.999650,140.0,376.468191,450.363472,0.387824,127.422660,243.041596,451.633110,2780.999650,1.0
1,2.0,140.0,513.247102,536.864699,1.068547,149.279603,336.853440,616.545553,2464.060520,140.0,384.935326,402.648525,0.801410,111.959702,252.640080,462.409165,1848.045390,2.0
2,3.0,140.0,742.758208,628.182745,1.725003,344.374113,520.265933,901.080894,2786.543099,140.0,371.379104,314.091373,0.862502,172.187056,260.132966,450.540447,1393.271550,3.0
3,4.0,140.0,922.672701,769.356046,38.373249,351.300392,660.338677,1453.806070,2865.829390,140.0,230.668175,192.339012,9.593312,87.825098,165.084669,363.451518,716.457347,4.0


## Write to CSV

In [15]:
# leave in indexes to preserve school name and seed
# this is pulled into QGIS and saved as geojson and converted to .json for proper formatting
seedsDist.to_csv('../data/cleaned/seeds-by-school.csv')
seedsMerged.to_csv('../data/cleaned/seeds-overall.csv', index=False)

# Aggregate by Conference

In [16]:
# calculate statistics at conference level
conf = data.groupby('conference').describe()

# merge weighted distance and unweighted distance in single dataframe
confAll = pd.merge(conf.distance, conf.weightedDist, 
             left_on=conf.distance.index.get_level_values('conference'), 
             right_on=conf.weightedDist.index.get_level_values('conference'),
             suffixes=('_dist', '_wtDist'))

confAll.head()

,key_0,count_dist,mean_dist,std_dist,min_dist,25%_dist,50%_dist,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
0,American Athletic Conference,35.0,544.058284,584.525520,27.304459,214.571640,348.141829,630.326868,2464.060520,35.0,390.199946,475.859494,10.968326,162.115023,261.106372,430.366276,2038.307527
1,Atlantic 10 Conference,11.0,969.521764,958.868496,45.877546,132.428889,288.715757,1895.191486,2278.013061,11.0,431.138401,464.632374,22.938773,68.080500,288.715757,550.229958,1365.835010
2,Atlantic Coast Conference,139.0,600.842046,703.710799,1.068547,127.422660,345.194626,658.086142,2865.829390,139.0,302.426192,313.246211,0.801410,87.444525,193.695540,401.404772,1547.355379
3,Big 12 Conference,78.0,544.761848,402.366177,17.777579,219.233497,454.134239,773.462956,1631.984883,78.0,299.547673,225.203496,10.635634,149.931525,226.338280,406.101337,1054.014359
4,Big East Conference,44.0,700.308612,803.608330,12.131191,234.536763,375.350575,765.596346,2863.666650,44.0,418.092775,584.856203,6.065595,114.869779,221.481390,397.323866,2780.999650


## Write to CSV

In [17]:
confAll.to_csv('../data/cleaned/conference-agg.csv')

# Playground - Work In Progress Below

In [18]:
tmp = data.groupby('conference').describe()
dist = tmp.distance
wtDist = tmp.weightedDist
wtDist

,count,mean,std,min,25%,50%,75%,max
conference,,,,,,,,
American Athletic Conference,35.0,390.199946,475.859494,10.968326,162.115023,261.106372,430.366276,2038.307527
Atlantic 10 Conference,11.0,431.138401,464.632374,22.938773,68.080500,288.715757,550.229958,1365.835010
Atlantic Coast Conference,139.0,302.426192,313.246211,0.801410,87.444525,193.695540,401.404772,1547.355379
Big 12 Conference,78.0,299.547673,225.203496,10.635634,149.931525,226.338280,406.101337,1054.014359
Big East Conference,44.0,418.092775,584.856203,6.065595,114.869779,221.481390,397.323866,2780.999650
Big Ten Conference,94.0,332.949736,342.701517,8.249110,101.621833,213.241358,445.067223,1806.430803
Missouri Valley Conference,2.0,167.684644,91.991297,102.636974,135.160809,167.684644,200.208479,232.732315
Mountain West Conference,13.0,293.173881,212.392758,57.787579,117.833624,285.453367,352.895410,844.102336
Pac-12 Conference,59.0,459.938865,427.517174,0.387824,174.191450,389.337839,587.074272,2211.465583


In [19]:
m = pd.merge(tmp.distance, tmp.weightedDist, 
               left_on=tmp.distance.index.get_level_values('conference'), 
               right_on=tmp.weightedDist.index.get_level_values('conference'),
               suffixes=('_dist', '_wtDist'))
m

,key_0,count_dist,mean_dist,std_dist,min_dist,25%_dist,50%_dist,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
0,American Athletic Conference,35.0,544.058284,584.525520,27.304459,214.571640,348.141829,630.326868,2464.060520,35.0,390.199946,475.859494,10.968326,162.115023,261.106372,430.366276,2038.307527
1,Atlantic 10 Conference,11.0,969.521764,958.868496,45.877546,132.428889,288.715757,1895.191486,2278.013061,11.0,431.138401,464.632374,22.938773,68.080500,288.715757,550.229958,1365.835010
2,Atlantic Coast Conference,139.0,600.842046,703.710799,1.068547,127.422660,345.194626,658.086142,2865.829390,139.0,302.426192,313.246211,0.801410,87.444525,193.695540,401.404772,1547.355379
3,Big 12 Conference,78.0,544.761848,402.366177,17.777579,219.233497,454.134239,773.462956,1631.984883,78.0,299.547673,225.203496,10.635634,149.931525,226.338280,406.101337,1054.014359
4,Big East Conference,44.0,700.308612,803.608330,12.131191,234.536763,375.350575,765.596346,2863.666650,44.0,418.092775,584.856203,6.065595,114.869779,221.481390,397.323866,2780.999650
5,Big Ten Conference,94.0,697.795391,704.472325,8.249110,151.786305,423.744431,915.353168,2768.812156,94.0,332.949736,342.701517,8.249110,101.621833,213.241358,445.067223,1806.430803
6,Missouri Valley Conference,2.0,670.738577,367.965190,410.547896,540.643237,670.738577,800.833918,930.929259,2.0,167.684644,91.991297,102.636974,135.160809,167.684644,200.208479,232.732315
7,Mountain West Conference,13.0,628.901540,466.985908,231.150318,352.895410,380.604490,925.244249,1688.204671,13.0,293.173881,212.392758,57.787579,117.833624,285.453367,352.895410,844.102336
8,Pac-12 Conference,59.0,811.178417,692.094786,0.387824,369.559704,533.457641,965.115478,2687.468752,59.0,459.938865,427.517174,0.387824,174.191450,389.337839,587.074272,2211.465583
9,Southeastern Conference,74.0,562.148034,481.868779,49.672095,233.860625,476.600342,645.488090,2181.977569,74.0,302.783505,277.098408,22.679792,123.202285,212.875135,413.586600,1710.254130


In [20]:
display(data.weightedDist.describe())
display(data.loc[data.weightedDist > 2700])

count     560.000000
mean      340.862699
std       358.548307
min         0.387824
25%       114.888579
50%       228.521879
75%       435.534435
max      2780.999650
Name: weightedDist, dtype: float64

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance,weightedDist
531,1.0,St. John's,"Long Beach, California",1986.0,1986531,St. John's University,Red Storm,Jamaica,New York,Private/Catholic,Big East Conference,St. John's University Jamaica New York,-73.990073,40.729944,POINT (-73.99007259999999 40.72994420000001),2780.99965,2780.99965


In [36]:
# display(data.distance.describe())
# display(data.loc[data.distance < 1])
display(data.loc[(data.distance > 370) & (data.distance < 380) & (data.seed == 1.0)])

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance,weightedDist
353,1.0,Minnesota,"Kansas City, MO",1997.0,1997353,University of Minnesota,Golden Gophers,Minneapolis–Saint Paul,Minnesota,State,Big Ten Conference,University of Minnesota St. Paul Minnesota,-93.179456,44.986339,POINT (-93.17945580218505 44.9863392),376.018034,376.018034


In [37]:
display(data.loc[(data.distance > 500) & (data.distance < 530) & (data.seed == 2.0)])

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance,weightedDist
6,2.0,Kentucky,"Jacksonville, FL",2019.0,20196,University of Kentucky,Wildcats,Lexington,Kentucky,State,Southeastern Conference,University of Kentucky Lexington Kentucky,-84.504820,38.027036,POINT (-84.50481984038481 38.02703575),509.968624,382.476468
261,2.0,Wake Forest,"Tampa, FL",2003.0,2003261,Wake Forest University,Demon Deacons,Winston-Salem,North Carolina,Private/Non-Sectarian,Atlantic Coast Conference,Wake Forest University Winston-Salem North Car...,-80.278115,36.134335,POINT (-80.2781152 36.134335),526.534804,394.901103


In [38]:
display(data.loc[(data.distance > 730) & (data.distance < 745) & (data.seed == 3.0)])

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance,weightedDist
249,3.0,Texas,"Denver, CO",2004.0,2004249,University of Texas at Austin,Longhorns,Austin,Texas,State,Big 12 Conference,University of Texas at Austin Austin Texas,-97.731956,30.284151,POINT (-97.7319559808022 30.284151),740.616118,370.308059


In [39]:
display(data.loc[(data.distance > 900) & (data.distance < 930) & (data.seed == 4.0)])

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference,address,lng,lat,geometry,distance,weightedDist
95,4.0,San Diego State,"Spokane, WA",2014.0,201495,San Diego State University,Aztecs,San Diego,California,State,Mountain West Conference,San Diego State University San Diego California,-117.073851,32.774816,POINT (-117.0738505256308 32.77481589999999),925.244249,231.311062
157,4.0,Wisconsin,"Jacksonville, FL",2010.0,2010157,University of Wisconsin–Madison,Badgers,Madison,Wisconsin,State,Big Ten Conference,University of Wisconsin–Madison Madison Wisconsin,-89.405939,43.070526,POINT (-89.40593869999999 43.0705257),926.712988,231.678247
447,4.0,Oklahoma,"Tempe, AZ",1992.0,1992447,University of Oklahoma,Sooners,Norman,Oklahoma,State,Big 12 Conference,University of Oklahoma Norman Oklahoma,-97.445708,35.195988,POINT (-97.44570827599857 35.1959878),907.522451,226.880613


In [22]:
# data.distance.plot()
data.weightedDist.describe()

count     560.000000
mean      340.862699
std       358.548307
min         0.387824
25%       114.888579
50%       228.521879
75%       435.534435
max      2780.999650
Name: weightedDist, dtype: float64

In [46]:
# df.sort_values(by=['col1'])
pd.set_option('display.max_columns', None)
schoolsSorted.sort_values(by=['mean_dist'])

,school_common_name,school_full_name,team,city,state,type,conference,address,lng,lat,key_0,count_dist,mean_dist,std_dist,min_dist,25%_dist,50%_dist,75%_dist,max_dist,count_wtDist,mean_wtDist,std_wtDist,min_wtDist,25%_wtDist,50%_wtDist,75%_wtDist,max_wtDist
83,DePaul,DePaul University,Blue Demons,Chicago,Illinois,Private/Catholic,Big East Conference,DePaul University Chicago Illinois,-87.654726,41.924020,DePaul,1.0,14.406784,NaN,14.406784,14.406784,14.406784,14.406784,14.406784,1.0,7.203392,NaN,7.203392,7.203392,7.203392,7.203392,7.203392
82,La Salle,La Salle University,Explorers,Philadelphia,Pennsylvania,Private/Catholic,Atlantic 10 Conference,La Salle University Philadelphia Pennsylvania,-75.154018,40.037470,La Salle,1.0,186.969348,NaN,186.969348,186.969348,186.969348,186.969348,186.969348,1.0,46.742337,NaN,46.742337,46.742337,46.742337,46.742337,46.742337
68,Alabama,University of Alabama,Crimson Tide,Tuscaloosa,Alabama,State,Southeastern Conference,University of Alabama Tuscaloosa Alabama,-87.539674,33.212082,Alabama,3.0,194.538449,142.894640,49.672095,124.119584,198.567073,266.971626,335.376179,3.0,112.809325,120.297037,37.254071,43.447920,49.641768,150.586951,251.532134
27,Butler,Butler University,Bulldogs,Indianapolis,Indiana,Private/Non-Sectarian,Big East Conference,Butler University Indianapolis Indiana,-86.173749,39.840719,Butler,1.0,226.329078,NaN,226.329078,226.329078,226.329078,226.329078,226.329078,1.0,56.582269,NaN,56.582269,56.582269,56.582269,56.582269,56.582269
3,Virginia,University of Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference,University of Virginia Charlottesville Virginia,-78.505500,38.041058,Virginia,6.0,233.604168,89.021514,140.746063,162.400271,227.362896,279.018491,369.170067,6.0,177.984456,72.930227,92.292517,140.746063,155.634117,213.152715,296.237023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,St. John's,St. John's University,Red Storm,Jamaica,New York,Private/Catholic,Big East Conference,St. John's University Jamaica New York,-73.990073,40.729944,St. John's,5.0,1803.529194,970.990010,636.894998,885.812198,2355.297484,2358.641641,2780.999650,5.0,1501.481643,1157.579618,159.223749,442.906099,1768.981231,2355.297484,2780.999650
85,VCU,Virginia Commonwealth University,Rams,Richmond,Virginia,State,Atlantic 10 Conference,Virginia Commonwealth University Richmond Virg...,-77.453064,37.548215,VCU,1.0,1821.113346,NaN,1821.113346,1821.113346,1821.113346,1821.113346,1821.113346,1.0,1365.835010,NaN,1365.835010,1365.835010,1365.835010,1365.835010,1365.835010
46,Saint Louis,Saint Louis University,Billikens,St. Louis,Missouri,Private/Catholic,Atlantic 10 Conference,Saint Louis University St. Louis Missouri,-90.231677,38.635284,Saint Louis,1.0,1969.269626,NaN,1969.269626,1969.269626,1969.269626,1969.269626,1969.269626,1.0,492.317407,NaN,492.317407,492.317407,492.317407,492.317407,492.317407
67,Dayton,University of Dayton,Flyers,Dayton,Ohio,Private/Catholic,Atlantic 10 Conference,University of Dayton Dayton Ohio,-84.179195,39.738460,Dayton,1.0,2123.826607,NaN,2123.826607,2123.826607,2123.826607,2123.826607,2123.826607,1.0,530.956652,NaN,530.956652,530.956652,530.956652,530.956652,530.956652


In [48]:
import geocoder as g

In [78]:
g.osm('west point highlands ny')

<[OK] Osm - Geocode [Sunrise Highway, Patchogue, Suffolk County, New York, 11772, United States of America]>

In [23]:
import matplotlib
%matplotlib inline

seed = data.groupby('seed').describe()
seed.distance
data.groupby('seed').distance.median()

seed
1.0    243.041596
2.0    336.853440
3.0    520.265933
4.0    660.338677
Name: distance, dtype: float64

In [24]:
school = data.groupby('school_common_name').describe()
school.distance.head()

# print (df.drop_duplicates(['Cat']))
d = data.drop_duplicates(['school_common_name']).drop(['distance', 'seed', 'site', 'id', 'geometry'], axis=1)
d

m = pd.merge(d, school.distance, how='left', left_on='school_common_name', right_on=school.distance.index.get_level_values('school_common_name'))
m

# df.sort_values(by=['col1'])
m.sort_values(by=['mean'])

# x = school.distance
# x['name'] = school.distance.index.get_level_values('school_common_name')
# # school.loc[school.distance, 'name'] = school.distance.index.get_level_values('school_common_name')
# x

# s = pd.merge(x, data, how='right', left_on='name', right_on='school_common_name')
# s
# # m = pd.merge(x, data, how='right', left_on='name', right_on='school_common_name')
# # # m = pd.merge(data, school.distance, how='left', left_on='school_common_name', right_on=school.distance.index.get_level_values('school_common_name'))
# # # m = m.drop(['distance'], axis=1)
# # m
# # # print(m)

# # m.to_json('../data/cleaned/school-mean.json', orient='records')
# # school.distance[school.distance.index.get_level_values('school_common_name') == 'Kentucky']

,school_common_name,year,school_full_name,team,city,state,type,conference,address,lng,lat,weightedDist,count,mean,std,min,25%,50%,75%,max
83,DePaul,1987.0,DePaul University,Blue Demons,Chicago,Illinois,Private/Catholic,Big East Conference,DePaul University Chicago Illinois,-87.654726,41.924020,7.203392,1.0,14.406784,NaN,14.406784,14.406784,14.406784,14.406784,14.406784
82,La Salle,1990.0,La Salle University,Explorers,Philadelphia,Pennsylvania,Private/Catholic,Atlantic 10 Conference,La Salle University Philadelphia Pennsylvania,-75.154018,40.037470,46.742337,1.0,186.969348,NaN,186.969348,186.969348,186.969348,186.969348,186.969348
68,Alabama,2002.0,University of Alabama,Crimson Tide,Tuscaloosa,Alabama,State,Southeastern Conference,University of Alabama Tuscaloosa Alabama,-87.539674,33.212082,251.532134,3.0,194.538449,142.894640,49.672095,124.119584,198.567073,266.971626,335.376179
27,Butler,2017.0,Butler University,Bulldogs,Indianapolis,Indiana,Private/Non-Sectarian,Big East Conference,Butler University Indianapolis Indiana,-86.173749,39.840719,56.582269,1.0,226.329078,NaN,226.329078,226.329078,226.329078,226.329078,226.329078
3,Virginia,2019.0,University of Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference,University of Virginia Charlottesville Virginia,-78.505500,38.041058,296.237023,6.0,233.604168,89.021514,140.746063,162.400271,227.362896,279.018491,369.170067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,St. John's,2000.0,St. John's University,Red Storm,Jamaica,New York,Private/Catholic,Big East Conference,St. John's University Jamaica New York,-73.990073,40.729944,1768.981231,5.0,1803.529194,970.990010,636.894998,885.812198,2355.297484,2358.641641,2780.999650
85,VCU,1985.0,Virginia Commonwealth University,Rams,Richmond,Virginia,State,Atlantic 10 Conference,Virginia Commonwealth University Richmond Virg...,-77.453064,37.548215,1365.835010,1.0,1821.113346,NaN,1821.113346,1821.113346,1821.113346,1821.113346,1821.113346
46,Saint Louis,2013.0,Saint Louis University,Billikens,St. Louis,Missouri,Private/Catholic,Atlantic 10 Conference,Saint Louis University St. Louis Missouri,-90.231677,38.635284,492.317407,1.0,1969.269626,NaN,1969.269626,1969.269626,1969.269626,1969.269626,1969.269626
67,Dayton,2003.0,University of Dayton,Flyers,Dayton,Ohio,Private/Catholic,Atlantic 10 Conference,University of Dayton Dayton Ohio,-84.179195,39.738460,530.956652,1.0,2123.826607,NaN,2123.826607,2123.826607,2123.826607,2123.826607,2123.826607


In [25]:
m.to_json('../data/cleaned/schools-default.json')
m.to_csv('../data/cleaned/schools-default.csv', index=False)


# with open('../data/cleaned/schools-default.json', 'w') as f:
#     f.write(m.to_json())

In [26]:
conf = data.groupby('conference').describe()
conf.distance

,count,mean,std,min,25%,50%,75%,max
conference,,,,,,,,
American Athletic Conference,35.0,544.058284,584.525520,27.304459,214.571640,348.141829,630.326868,2464.060520
Atlantic 10 Conference,11.0,969.521764,958.868496,45.877546,132.428889,288.715757,1895.191486,2278.013061
Atlantic Coast Conference,139.0,600.842046,703.710799,1.068547,127.422660,345.194626,658.086142,2865.829390
Big 12 Conference,78.0,544.761848,402.366177,17.777579,219.233497,454.134239,773.462956,1631.984883
Big East Conference,44.0,700.308612,803.608330,12.131191,234.536763,375.350575,765.596346,2863.666650
Big Ten Conference,94.0,697.795391,704.472325,8.249110,151.786305,423.744431,915.353168,2768.812156
Missouri Valley Conference,2.0,670.738577,367.965190,410.547896,540.643237,670.738577,800.833918,930.929259
Mountain West Conference,13.0,628.901540,466.985908,231.150318,352.895410,380.604490,925.244249,1688.204671
Pac-12 Conference,59.0,811.178417,692.094786,0.387824,369.559704,533.457641,965.115478,2687.468752


In [27]:
def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    
    weights = {1: 1, 2: 0.75, 3: 0.5, 4: 0.25}

#     print(weights)
#     print(str(d))
    try:
        return (weights[d] * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

data.groupby(['school_common_name']).apply(wavg, 'seed', 'distance')
# tmp = data.groupby(['school_common_name', 'seed']).describe()

#In [12]: df.iloc[df.index.get_level_values('A') == 1]
# display(tmp.distance[tmp.distance.index.get_level_values('school_common_name') == 'Kentucky'])
# school.distance[school.distance.index.get_level_values('school_common_name') == 'Kentucky']
# tmp.distance


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
# ((450*10*1) + (381 * 7 * 0.75) + (628*2*0.5) + (535*3*0.25)) / 22

# [tmp.distance[tmp.distance.index.get_level_values('school_common_name') == i] for i in tmp.distance.index.get_level_values('school_common_name')]
seed = tmp.distance.index.get_level_values('seed')
school = tmp.distance.index.get_level_values('school_common_name')
# for i in school:
#      if 1 not in seed[school == i]:
#         s3 = pd.Series([4, 5, 6], index=[3, 4, 5])
#         df = pd.Series([0,0,0,0,0,0,0,0], index=[i, 1])
#         print(df)

#             dfObj.loc['k'] = ['Smriti', 26, 'Bangalore', 'India']
#         tmp.distance.loc['1'] = [0,0,0,0,0,0,0,0]
#         df = pd.DataFrame([0], columns=list('count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max')) 
#         df = pd.DataFrame({'count': 0, 'mean': 0, 'std': 0, 'min': 0, '25%': 0, '50%': 0, '75%': 0, 'max': 0})
# modDfObj = dfObj.append({'Name' : 'Sahil' , 'Age' : 22} , ignore_index=True)
#         tmp.distance.append({'count': 0, 'mean': 0, 'std': 0, 'min': 0, '25%': 0, '50%': 0, '75%': 0, 'max': 0}, ignore_index=False)
#         print(seed[school == i])
# tmp.distance

x = tmp.distance
x['school'] = x.index.get_level_values('school_common_name')
x['seed'] = x.index.get_level_values('seed')
x.loc[x.school == 'Alabama']

In [ ]:
display(tmp.distance[tmp.distance.index.get_level_values('school_common_name') == 'Duke'])
school.distance[school.distance.index.get_level_values('school_common_name') == 'Duke']

In [ ]:
((14*142*1)+(11*270*0.75)+(5*843*.5)+(1*591*0.25)) / 31

In [ ]:
tmp.distance[tmp.distance.index.get_level_values('school_common_name') == 'Kansas']

In [ ]:
tmp.distance[tmp.distance.index.get_level_values('school_common_name') == 'North Carolina']

In [ ]:
tmp.distance[tmp.distance.index.get_level_values('school_common_name') == 'Michigan State']